In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import re
from contracted_basis import CAO_basis, shell_pair
from basic_operations import operation as op
from fast_multipole_method import fmm_box

In [2]:
#testcase6: artifical system C2H2


ERIs = []
fr = open("C2H2.out", 'r')
while True:
    line = fr.readline()
    if line == "":
        break
    
    if line.find("Two-Electron Integrals") != -1:
        while True:
            line = fr.readline()
            if line == "\n":
                break
            ERIs.append(float(re.split("[,()|]|\s ", line)[7]))
            
fr.close()

In [3]:
len(ERIs)

20736

In [4]:
(1e-10 / np.sqrt(max(ERIs)))**2

2.8233434032581385e-21

In [5]:
for i in range(0, 12):
    for j in range(0, 12):
        if ERIs[i*(12**3) + j*(12*12) + i*12 + j] > 2.82e-21:
            print([i,j])

[0, 0]
[0, 1]
[0, 2]
[0, 3]
[0, 4]
[0, 5]
[1, 0]
[1, 1]
[1, 2]
[1, 3]
[1, 4]
[1, 5]
[2, 0]
[2, 1]
[2, 2]
[2, 3]
[2, 4]
[2, 5]
[3, 0]
[3, 1]
[3, 2]
[3, 3]
[3, 4]
[3, 5]
[4, 0]
[4, 1]
[4, 2]
[4, 3]
[4, 4]
[4, 5]
[5, 0]
[5, 1]
[5, 2]
[5, 3]
[5, 4]
[5, 5]
[6, 6]
[6, 7]
[6, 8]
[6, 9]
[6, 10]
[6, 11]
[7, 6]
[7, 7]
[7, 8]
[7, 9]
[7, 10]
[7, 11]
[8, 6]
[8, 7]
[8, 8]
[8, 9]
[8, 10]
[8, 11]
[9, 6]
[9, 7]
[9, 8]
[9, 9]
[9, 10]
[9, 11]
[10, 6]
[10, 7]
[10, 8]
[10, 9]
[10, 10]
[10, 11]
[11, 6]
[11, 7]
[11, 8]
[11, 9]
[11, 10]
[11, 11]


In [6]:
C1_x = np.array([1.09294e+01, -5.46669e-01, 1.70075e-05])
H1_x = np.array([1.10290e+01, -1.74631e+00, -1.66212e+00])
C2_x = np.array([-1.09294e+01, 5.46669e-01, 4.72432e-05])
H2_x = np.array([-1.10289e+01, 1.74633e+00, 1.66218e+00])
atom_x_scale = np.vstack((C1_x, H1_x, C2_x, H2_x))

In [7]:
atom_x

NameError: name 'atom_x' is not defined

In [ ]:
[atom_x_scale, scale_factor] = op.cartesian_scaling_to_unit_range(atom_x)
atom_x_scale

In [7]:
basis = np.ndarray(12, dtype=CAO_basis)
basis[0] = CAO_basis(atom_x_scale[0], "C", 1, np.array([0, 0, 0]))
basis[1] = CAO_basis(atom_x_scale[0], "C", 2, np.array([0, 0, 0]))
basis[2] = CAO_basis(atom_x_scale[0], "C", 2, np.array([1, 0, 0]))
basis[3] = CAO_basis(atom_x_scale[0], "C", 2, np.array([0, 1, 0]))
basis[4] = CAO_basis(atom_x_scale[0], "C", 2, np.array([0, 0, 1]))
basis[5] = CAO_basis(atom_x_scale[1], "H", 1, np.array([0, 0, 0]))
basis[6] = CAO_basis(atom_x_scale[2], "C", 1, np.array([0, 0, 0]))
basis[7] = CAO_basis(atom_x_scale[2], "C", 2, np.array([0, 0, 0]))
basis[8] = CAO_basis(atom_x_scale[2], "C", 2, np.array([1, 0, 0]))
basis[9] = CAO_basis(atom_x_scale[2], "C", 2, np.array([0, 1, 0]))
basis[10] = CAO_basis(atom_x_scale[2], "C", 2, np.array([0, 0, 1]))
basis[11] = CAO_basis(atom_x_scale[3], "H", 1, np.array([0, 0, 0]))

In [8]:
ERIs_by_fmm = np.zeros(36*36)
ERIs_far_component = np.zeros(36*36)
p = 25
for i in range(0,6):
    for j in range(0,6):
        for k in range(6, 12):
            for l in range(6, 12):
                print([i,j,k,l])
                ERIs_far_component[i*(6**3)+j*(6**2)+(k-6)*6+(l-6)] = ERIs[i*(12**3)+j*(12**2)+k*12+l]
                box1 = fmm_box(np.array([9, 0, 0]))
                box2 = fmm_box(np.array([-9, 0, 0]))
                a = shell_pair(basis[i], basis[j])
                b = shell_pair(basis[k], basis[l])
                a.M_expansion_to_box(box1, p)
                b.M_expansion_to_box(box2, p)
                box1.box_interaction(box2)
                ERIs_by_fmm[i*(6**3)+j*(6**2)+(k-6)*6+(l-6)] = a.Mlm.product(box1.Llm).sum().real
                

[0, 0, 6, 6]
[0, 0, 6, 7]
[0, 0, 6, 8]
[0, 0, 6, 9]
[0, 0, 6, 10]
[0, 0, 6, 11]
[0, 0, 7, 6]
[0, 0, 7, 7]
[0, 0, 7, 8]
[0, 0, 7, 9]
[0, 0, 7, 10]
[0, 0, 7, 11]
[0, 0, 8, 6]
[0, 0, 8, 7]
[0, 0, 8, 8]
[0, 0, 8, 9]
[0, 0, 8, 10]
[0, 0, 8, 11]
[0, 0, 9, 6]
[0, 0, 9, 7]
[0, 0, 9, 8]
[0, 0, 9, 9]
[0, 0, 9, 10]
[0, 0, 9, 11]
[0, 0, 10, 6]
[0, 0, 10, 7]
[0, 0, 10, 8]
[0, 0, 10, 9]
[0, 0, 10, 10]
[0, 0, 10, 11]
[0, 0, 11, 6]
[0, 0, 11, 7]
[0, 0, 11, 8]
[0, 0, 11, 9]
[0, 0, 11, 10]
[0, 0, 11, 11]
[0, 1, 6, 6]
[0, 1, 6, 7]
[0, 1, 6, 8]
[0, 1, 6, 9]
[0, 1, 6, 10]
[0, 1, 6, 11]
[0, 1, 7, 6]
[0, 1, 7, 7]
[0, 1, 7, 8]
[0, 1, 7, 9]
[0, 1, 7, 10]
[0, 1, 7, 11]
[0, 1, 8, 6]
[0, 1, 8, 7]
[0, 1, 8, 8]
[0, 1, 8, 9]
[0, 1, 8, 10]
[0, 1, 8, 11]
[0, 1, 9, 6]
[0, 1, 9, 7]
[0, 1, 9, 8]
[0, 1, 9, 9]
[0, 1, 9, 10]
[0, 1, 9, 11]
[0, 1, 10, 6]
[0, 1, 10, 7]
[0, 1, 10, 8]
[0, 1, 10, 9]
[0, 1, 10, 10]
[0, 1, 10, 11]
[0, 1, 11, 6]
[0, 1, 11, 7]
[0, 1, 11, 8]
[0, 1, 11, 9]
[0, 1, 11, 10]
[0, 1, 11, 11]
[0, 2, 6, 6]
[0,

In [9]:
len(ERIs_far_component)

1296

In [10]:
ERIs_by_fmm

array([ 0.04569162,  0.01134829,  0.00014982, ...,  0.01217069,
        0.01691105,  0.04428981])

In [11]:
ERIs_far_component

array([ 0.045691  ,  0.011348  ,  0.00014982, ...,  0.012171  ,
        0.016911  ,  0.044289  ])

In [40]:
error = np.abs(ERIs_by_fmm-ERIs_far_component) / np.abs(ERIs_far_component)

In [38]:
error = ERIs_by_fmm-ERIs_far_component / ERIs_far_component

In [41]:
sum(error) / len(error)

0.00011134421569737757

In [14]:
for m in range(0, len(error)):
    if error[m] > 1e-5:
        l = m % 6 + 6
        k = (m // 6) % 6 + 6
        j = (m // 6 // 6) % 6
        i = (m // 6 // 6 // 6) % 6
        print([i, j, k, l], error[m])

[0, 0, 6, 6] 1.34660329831e-05
[0, 0, 6, 7] 2.51239553744e-05
[0, 0, 6, 8] 2.33643634749e-05
[0, 0, 6, 9] 1.29471222808e-05
[0, 0, 7, 6] 2.51239553744e-05
[0, 0, 7, 7] 5.25794226694e-05
[0, 0, 7, 8] 4.34109168125e-05
[0, 0, 7, 9] 1.87367765521e-05
[0, 0, 7, 10] 3.87765580194e-05
[0, 0, 7, 11] 4.96446649816e-05
[0, 0, 8, 6] 2.33643634746e-05
[0, 0, 8, 7] 4.34109168125e-05
[0, 0, 8, 8] 1.27588334132e-05
[0, 0, 8, 9] 1.42894114954e-05
[0, 0, 8, 10] 3.61343398183e-05
[0, 0, 8, 11] 0.00228541260482
[0, 0, 9, 6] 1.29471222809e-05
[0, 0, 9, 7] 1.87367765521e-05
[0, 0, 9, 8] 1.42894114954e-05
[0, 0, 9, 10] 2.91051078318e-05
[0, 0, 9, 11] 2.87295132777e-05
[0, 0, 10, 7] 3.87765580194e-05
[0, 0, 10, 8] 3.61343398183e-05
[0, 0, 10, 9] 2.91051078318e-05
[0, 0, 10, 10] 1.54083017519e-05
[0, 0, 11, 7] 4.96446649818e-05
[0, 0, 11, 8] 0.00228541260482
[0, 0, 11, 9] 2.87295132777e-05
[0, 1, 6, 6] 2.51239553749e-05
[0, 1, 6, 7] 4.912642927e-05
[0, 1, 6, 9] 2.01450610724e-05
[0, 1, 6, 10] 1.09871020185e-

In [42]:
for m in range(0, len(error)):
    if abs_error[m] > 1e-10:
        l = m % 6 + 6
        k = (m // 6) % 6 + 6
        j = (m // 6 // 6) % 6
        i = (m // 6 // 6 // 6) % 6
        print([i, j, k, l], abs_error[m])

[0, 0, 6, 6] 6.1527651303e-07
[0, 0, 6, 7] 2.85106645589e-07
[0, 0, 6, 8] 3.50044893581e-09
[0, 0, 6, 11] 2.11390401251e-08
[0, 0, 7, 6] 2.85106645589e-07
[0, 0, 7, 7] 2.40240640119e-06
[0, 0, 7, 8] 7.59170113217e-08
[0, 0, 7, 9] 1.63898079212e-09
[0, 0, 7, 11] 1.11263623157e-06
[0, 0, 8, 6] 3.50044893576e-09
[0, 0, 8, 7] 7.59170113217e-08
[0, 0, 8, 8] 5.84201464322e-07
[0, 0, 8, 9] 1.03722551222e-10
[0, 0, 8, 11] 6.8525811543e-07
[0, 0, 9, 7] 1.63898079212e-09
[0, 0, 9, 8] 1.03722551222e-10
[0, 0, 9, 9] 3.40219140416e-07
[0, 0, 9, 11] 3.56274694156e-07
[0, 0, 10, 10] 7.03281116864e-07
[0, 0, 10, 11] 1.37386304732e-07
[0, 0, 11, 6] 2.11390401243e-08
[0, 0, 11, 7] 1.11263623157e-06
[0, 0, 11, 8] 6.8525811543e-07
[0, 0, 11, 9] 3.56274694156e-07
[0, 0, 11, 10] 1.37386304732e-07
[0, 0, 11, 11] 2.3921714612e-07
[0, 1, 6, 6] 2.85106645594e-07
[0, 1, 6, 7] 1.38457928255e-07
[0, 1, 6, 8] 1.99220705721e-10
[0, 1, 6, 11] 2.16262052583e-08
[0, 1, 7, 6] 1.38457928255e-07
[0, 1, 7, 7] 7.28970566154

In [49]:
abs_error = np.abs(ERIs_by_fmm-ERIs_far_component)

In [50]:
sum(abs_error) / len(abs_error)

8.2693001555601284e-08

In [79]:
i=0; j=0; k=7; l=11
box1 = fmm_box(np.array([10, -1, -0.8]))
box2 = fmm_box(np.array([-10, 1, 0.8]))
p = 40
a = shell_pair(basis[i], basis[j])
b = shell_pair(basis[k], basis[l])
a.M_expansion_to_box(box1, p)
b.M_expansion_to_box(box2, p)
box1.box_interaction(box2)
box2.box_interaction(box1)

In [80]:
a.Mlm.product(box1.Llm).sum().real 

0.02241311263623157

In [81]:
ERIs[i*(12**3) + j*(12*12) + k*12 + l] 

0.022412

In [56]:
from fmm_source import gs_q_dist

In [57]:
c_k = np.exp( - sum((basis[i].x - basis[j].x)**2)*a.a_k )
c_k = c_k.reshape(9) * a.d.reshape(9)
c_a = a.a_p.reshape(9) 
d_k = np.exp( - sum((basis[k].x - basis[l].x)**2)*b.a_k )
d_k = d_k.reshape(9) * b.d.reshape(9)
d_a = b.a_p.reshape(9) 

In [58]:
c = np.ndarray(9, dtype=gs_q_dist)
d = np.ndarray(9, dtype=gs_q_dist)
for m in range(0, 9):
    c[m] = gs_q_dist(a.X_p[m], c_a[m], c_k[m])
    d[m] = gs_q_dist(b.X_p[m], d_a[m], d_k[m])

In [59]:
direct_sum = 0
for i in range(0, 9):
    for j in range(0, 9):
        direct_sum += c[i].near_field_interaction(d[j], 1)

In [60]:
direct_sum 

0.044921625208341989

In [ ]:
b.d.reshape(9)

In [ ]:
np.exp( - sum((basis[k].x - basis[l].x)**2)*b.a_k)

In [ ]:
b.a_k

In [ ]:
scale_factor = [0,1]

In [ ]:
b.Mlm.V_matrix

In [ ]:
a.Mlm.V_matrix

In [ ]:
b.X_p

In [ ]:
i=0; j=0; k=6; l=6
a = shell_pair(basis[i], basis[j])
b = shell_pair(basis[k], basis[l])
box1 = fmm_box(np.array([0.95, 0.5, 0.5]))
box2 = fmm_box(np.array([0.05, 0.5, 0.5]))
p = 2
a.M_expansion_to_box(box1, p)
b.M_expansion_to_box(box2, p)
box1.box_interaction(box2)
box2.box_interaction(box1)

In [ ]:
a.Mlm.product(box1.Llm).sum().real 

In [ ]:
b.Mlm.V_matrix 

In [ ]:
a.Mlm.V_matrix

In [ ]:
b.X_p